In [1]:
from data import *

trainset, devset, testset = get_dataset()

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fangyuanyu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import random

gqa = giveQA() # Initialize Model
k = random.randint(0, len(trainset)-1) # Random Index
pred = gqa(trainset[k]['question'], '') # Make Predictions
correct_pred = check_correctness(trainset[k], pred) # Check Correctness


evaluate(gqa, devset=devset)

Average Correctness: 0.8333333333333334


In [14]:
import ast

agent_template = """
<Role> You are a customer, engaging in a conversation with FWD insurance sales agent. <\Role>

<Objective> You are trying to decide whether the insurance product suits your need based on the conversation. <\Objective>

<Requirement> Follow the following requirements:
  {requirements}
<\Requirement>
"""

def format_chromosome(genes):
    return "\n  ".join(genes)

requirements = ["Stick to your role", "Act like you are poor"]

agent_script = agent_template.format(requirements=format_requirements(requirements))
print(agent_script)


<Role> You are a customer, engaging in a conversation with FWD insurance sales agent. <\Role>

<Objective> You are trying to decide whether the insurance product suits your need based on the conversation. <\Objective>

<Requirement> Follow the following requirements:
  Stick to your role
  Act like you are poor
<\Requirement>



In [5]:
gqa = giveQA() # Initialize Model
# Data is not included into the teleprompter object, only the metric (not even the module is included ...)
teleprompter = BootstrapFewShotWithRandomSearch(metric = check_correctness, max_bootstrapped_demos=1, num_candidate_programs=2)

teleprompter.compile(student = gqa, teacher = gqa, trainset=trainset, valset=devset)

# evaluate(cited_longformqa)

Going to sample between 1 and 1 traces per predictor.
Will attempt to train 2 candidate sets.





















Average Metric: 15 / 18  (83.3): 100%|██████████| 18/18 [00:00<00:00, 1665.62it/s]


Average Metric: 15 / 18  (83.3%)
Score: 83.33 for set: [0]
New best score: 83.33 for seed -3
Scores so far: [83.33]
Best score: 83.33


Error for example in dev set: 		 This model's maximum context length is 4097 tokens. However, you requested 4435 tokens (3435 in the messages, 1000 in the completion). Please reduce the length of the messages or completion.
Error for example in dev set: 		 This model's maximum context length is 4097 tokens. However, you requested 4302 tokens (3302 in the messages, 1000 in the completion). Please reduce the length of the messages or completion.
Error for example in dev set: 		 This model's maximum context length is 4097 tokens. However, you requested 4356 tokens (3356 in the messages, 1000 in the completion). Please reduce the length of the messages or completion.


Error for example in dev set: 		 This model's maximum context length is 4097 tokens. However, you requested 4285 tokens (3285 in the messages, 1000 in the completion). Please reduce the length of the messages or completion.


InvalidRequestError: This model's maximum context length is 4097 tokens. However, you requested 4302 tokens (3302 in the messages, 1000 in the completion). Please reduce the length of the messages or completion.

In [ ]:
longformqa = LongFormQA()
teleprompter = BootstrapFewShotWithRandomSearch(metric = answer_correctness, max_bootstrapped_demos=2, num_candidate_programs=6)
cited_longformqa_teacher = teleprompter.compile(student=longformqa, teacher = assert_transform_module(LongFormQAWithAssertions().map_named_predictors(Retry), backtrack_handler), trainset=trainset, valset=devset[:100])
evaluate(cited_longformqa_teacher)